In [ ]:
from pandas import Series
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Функции, использующиеся при обработке данных


# Функция для расчёта статистических параметров: iqr, квартили, границы выбросов, количество выбросов
# Передаём на вход столбец DataFrame
def quantile_info(column):
    # Перцентили и IQR
    perc25 = column.quantile(0.25)
    perc75 = column.quantile(0.75)
    iqr = perc75 - perc25
    # Границы выбросов
    low = perc25 - 1.5*iqr
    high = perc75 + 1.5*iqr
    # Количество выбросов в столбце
    count_low = (column[(column<low)]).count()
    count_high = (column[(column>high)]).count()
    # Печатаем вычисленные значения на экране    
    print(
        f'25-й перцентиль: {perc25}\n',
        f'75-й перцентиль: {perc75}\n',
        f'IQR: {iqr}\n',
        f'Границы выбросов: [{low}, {high}]\n',
        f'Количество выбросов вниз: {count_low}\n',
        f'Количество выбросов вверх: {count_high}',
        sep='')

    
# Функция для замены выбросов медианным значением
def change_on_median(column):
    # Перцентили и IQR
    perc25 = column.quantile(0.25)
    perc50 = column.quantile(0.5)
    perc75 = column.quantile(0.75)
    iqr = perc75 - perc25
    # Границы выбросов
    low = perc25 - 1.5*iqr
    high = perc75 + 1.5*iqr
    
    return column.apply(lambda x: perc50 if (x < low) | (x > high) else x)


# Функция, разделяющая датасет на обучающую и валидационную части со стандартизацией числовых признаков
# Принимает на вход DataFrame, возвращает 4 массива numpy
def form(DataFrame):
    y = DataFrame['default'].values
    x = DataFrame.drop(['default'], axis=1)
    
    x_train_pd, x_valid_pd, y_train, y_valid = train_test_split(x, y, test_size=0.20, random_state=42)
    
    # Cтандартизируем числовые признаки
    scaler = StandardScaler()
    x_train_num = scaler.fit_transform(x_train_pd[num_cols].values)
    x_valid_num = scaler.transform(x_valid_pd[num_cols].values)
    
    # И соберём все признаки в numpy array
    x_train = np.hstack([x_train_num, x_train_pd.drop(columns=num_cols).values])
    x_valid = np.hstack([x_valid_num, x_valid_pd.drop(columns=num_cols).values])
    
    return x_train, x_valid, y_train, y_valid


# Функция, выводящая на экран график ROC_AUC
def print_roc_auc(val, pred):
    fpr, tpr, threshold = roc_curve(val, pred)
    roc_auc = roc_auc_score(val, pred)
    
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()
    
    
# Функция, выводящая на экран значения метрик логистической регрессии
def print_metrics(val, pred, acc=0):
    # Выбираем, печатать ли метрику accuracy
    if acc == 1:
        print('accuracy:', '%0.4f' %accuracy_score(val, pred))
        
    print('balanced_accuracy:', '%0.4f' %balanced_accuracy_score(val, pred))
    print('precision_score:', '%0.4f' %precision_score(val, pred))
    print('recall_score:', '%0.4f' %recall_score(val, pred))
    print('f1_score:', '%0.4f' %f1_score(val, pred))

## Загрузка начальных данных

In [ ]:
# Загружаем таблицу c данными клиентов банка
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
# Загружаем данные для обучения модели
train = pd.read_csv(DATA_DIR+'/train.csv')
# Загружаем данные для теста модели
test = pd.read_csv(DATA_DIR+'/test.csv')
# Загружаем файл с итоговыми данными
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
# описание данных

# client_id - идентификатор клиента
# education - уровень образования
# sex - пол заемщика
# age - возраст заемщика
# car - флаг наличия автомобиля
# car_type - флаг автомобиля иномарки
# deсline_app_cnt - количество отказанных прошлых заявок
# good_work - флаг наличия “хорошей” работы
# bki_request_cnt - количество запросов в БКИ
# home_address - категоризатор домашнего адреса
# work_address - категоризатор рабочего адреса
# income - доход заемщика
# foreign_passport - наличие загранпаспорта
# sna - связь заемщика с клиентами банка
# first_time - давность наличия информации о заемщике
# score_bki - скоринговый балл по данным из БКИ
# region_rating - рейтинг региона
# app_date - дата подачи заявки
# default - флаг дефолта по кредиту

In [ ]:
# Смотрим исходные данные
display(train.head(10))
display(train.info())

In [ ]:
display(test.head(10))
display(test.info())

In [ ]:
display(sample_submission.head(10))
display(sample_submission.info())

In [ ]:
# Для корректной обработки признаков объединяем обучающую и тестовую выборки в один датасет
train['sample'] = 1 # помечаем, где обучающая выборка
test['sample'] = 0 # помечаем, где тестовая выборка
# в тесте у нас нет значения default, мы его должны предсказать, 
# поэтому пока заполним произвольными значениями.
# Пусть каждый седьмой клиент имеет default=1
test['default'] = test['client_id'].apply(lambda x: 1 if x%7 == 0 else 0) 

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# Информация по объединённому датасету
data.info()

Теперь на основе этих данных построим наивную модель логистической регрессии, для понимания, какого результата можно добиться с минимальной обработкой данных.

## Обработка данных для наивной модели

In [ ]:
# Скопируем датасет
data_naiv = data.copy()

In [ ]:
# В первую очередь посмотрим распределение целевой переменной default в обучающей выборке:
labels = train.default.value_counts().index
values = train.default.value_counts().values

plt.title('Разбивка клиентов по "default"')
plt.pie(values, labels=labels, explode=[0.3, 0], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

Как видно, клиентов с дефолтом значительно меньше, чем клиентов без дефолта. Поэтому в дальнейшем при обучении модели нужно будет применять методы для несбалансированных классов.

In [ ]:
# Смотрим на количество пропусков по столбцам
data_naiv.isna().sum()

In [ ]:
# Как мы видим, пропуски встречаются только в информации об образовании клиентов.
# Посмотрим на признак поближе.
data_naiv['education'].value_counts(dropna=False).plot.barh()

plt.show()

Что значат уровни образования:

SCH - school - те, у кого среднее образование (только школа).

UGR - undergraduate - бакалавры.

GRD - магистры

PGR - postgraduate - учёная степень PhD (кандидаты наук)

ACD - высший уровень.

Пропусков немного относительно общего количества записей (~0.4%), поэтому присоединим их 
к самой многочисленной категории SCH.

In [ ]:
data_naiv['education'] = data_naiv['education'].fillna('SCH')
# Проверяем количество пропусков по столбцам
data_naiv.isna().sum()

In [ ]:
# Категориальные и числовые признаки обрабатываются по-разному.
# Посмотрим, к каким категориям какие признаки относятся.
for a in data_naiv.columns:
    k = data_naiv[a].nunique()
    print(f'Количество значений признаков в столбце {a}: {k}')
    if a == 'sex':
        print('(В современном безумном мире мы должны проверить)')
    print(f'Тип значений признаков в столбце {a}: {data_naiv[a].dtypes}\n\n')
    

Таким образом, исходя из типа данных и их описания, мы можем разделить признаки на категории следующим образом:

- категориальные - education, region_rating, home_address, work_address, sna, first_time
- бинарные - sex, car, car_type, good_work, foreign_passport
- числовые - age, decline_app_cnt, score_bki, bki_request_cnt, income

Отдельный случай со столбцом app_date, так как там содержится дата. В наивной модели не будем его учитывать

In [ ]:
# Создадим списки колонок по категориям для удобства работы.
# Столбцы default и client_id не включаем, так как первый - целевая переменная,
# а второй - просто номер клиента
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']

# Удаляем столбец app_date и sample
data_naiv = data_naiv.drop(['app_date', 'sample'], axis=1)

In [ ]:
# Для начала переведём бинарные признаки в числа с помощью LabelEncoder
label_encoder = LabelEncoder()

for b in bin_cols:
    data_naiv[b] = label_encoder.fit_transform(data_naiv[b])
    
# убедимся в преобразовании    
data_naiv.head()

In [ ]:
# Посмотрим на распределение числовых переменных:
for c in num_cols:
    data_naiv[c].plot.hist(bins=26)
    plt.title(c)
    plt.show()

После построения гистограмм стало очевидно, что распределения почти всех числовых переменных имеют тяжёлый правый хвост.

Для того, чтобы сделать распределение данных переменных более нормальным, можно работать с логарифмированными величинами этих переменных. Тогда мы сможем избежать чувствительности к сильным отклонениям в суммах у линейных моделей.

In [ ]:
# прологарифмируем все числовые столбцы, кроме score_bki
for d in num_cols:
    data_naiv[d] = data_naiv[d].apply(lambda x: np.log(x + 1) if d != 'score_bki' else x)
    data_naiv[d].plot.hist(bins=26)
    plt.title(d)
    plt.show()


In [ ]:
# Закодируем категориальные признаки с помощью OneHotEncoding
X_naiv_cat =  pd.get_dummies(data_naiv, columns=cat_cols).values
X_naiv_cat.shape

In [ ]:
# Объединим стандартизованные числовые, бинарные и закодированные категориальные
# переменные в одно признаковое пространство, разделив при этом признаки
# и целевую переменную.
X_naiv = np.hstack([data_naiv[num_cols].values, data_naiv[bin_cols].values, X_naiv_cat])
Y_naiv = data_naiv['default'].values

In [ ]:
# Разделим данные для обучения на тренировочную и тестовую выборки следующим образом:
X_train_naiv, X_test_naiv, Y_train_naiv, Y_test_naiv = train_test_split(X_naiv, Y_naiv, test_size=0.20, random_state=42)

In [ ]:
# Обучаем модель на стандартных настройках логистической регрессии
# Логистическая регрессия
model_naiv = LogisticRegression(max_iter=5000)
model_naiv.fit(X_train_naiv, Y_train_naiv)

probs_naiv = model_naiv.predict_proba(X_test_naiv)
Y_pred_naiv = model_naiv.predict(X_test_naiv)
probs_naiv = probs_naiv[:,1]

# Выводим на экран график roc_auc
print_roc_auc(Y_test_naiv, probs_naiv)

In [ ]:
# Матрица ошибок для нашего алгоритма
print(confusion_matrix(Y_test_naiv, Y_pred_naiv))

Наша наивная модель все примеры причислила к классу default=0. Что вполне закономерно для такого разбаланса классов.

Посмотрим остальные метрики качества для логистической регрессии

In [ ]:
print_metrics(Y_test_naiv, Y_pred_naiv, acc=1)

Так как наша модель не дала значений TN и FN, метрики precision, recall и f1 не посчитались.
А вот метрика accuracy показала очень хорошее значение, что лишний раз подтверждает, что её нельзя использовать в случае несбалансированных классов. Поэтому в дальнейшем будем смотреть характеристику balanced_accuracy, которая гораздо лучше подходит для таких случаев.

Как видим, наивная модель не сильно отличается от случайного угадывания. Для улучшения характеристик модели необходимо проводить исследование и обработку данных.

## Анализ данных

In [ ]:
# Вспоним, как выглядит объединённый датасет
display(data.head(10))
display(data.info())

**Признак client_id**

In [ ]:
# Посмотрим, сколько значений в этом столбце
data['client_id'].nunique()

Получается, здесь просто порядковый номер клиента. Никакой обработки для этого столбца не предполагается.

**Признак app_date**

In [ ]:
# Здесь содержится дата заявки клиента в виде строки. Преобразуем эти данные в формат даты
data['app_date'] = pd.to_datetime(data['app_date'])
data['app_date'].value_counts()

На основе даты создадим новые признаки.

In [ ]:
# Посмотрим самую раннюю и самую позднюю даты в этом признаке
print(f"Самая ранняя дата: {data['app_date'].min()}")
print(f"Самая поздняя дата: {data['app_date'].max()}")

Все даты в 2014 году. Поэтому признак с годом создавать не будем, создадим следующие признаки:

номер месяца, номер дня недели, номер дня месяца, номер дня года

In [ ]:
data['month'] = data['app_date'].apply(lambda t: t.month)
data['day'] = data['app_date'].apply(lambda t: t.day)
data['weekday'] = data['app_date'].apply(lambda t: t.isoweekday())
data['yearday'] = data['app_date'].apply(lambda t: t.dayofyear)

# И удалим признак app_date
data = data.drop(['app_date'], axis=1)

In [ ]:
# Добавим вновь созданные признаки в списки колонок для облегчения работы
# 'weekday' к категориальным
cat_cols.append('weekday')

# Остальные к числовым
num_cols.append('month')
num_cols.append('day')
num_cols.append('yearday')

### Теперь обработаем числовые признаки
К числовым мы отнесли следующие признаки:

num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

Как мы помним, почти все признаки имеют сильное смещение. Поэтому для нормализации сначала прологарифмируем их, потом будем искать выбросы.

**Признак age**

In [ ]:
data['age'] = data['age'].apply(lambda x: np.log(x + 1))

data['age'].hist(bins=100)
plt.show()

quantile_info(data['age'])

Выбросы отсутствуют.

**Признак decline_app_cnt**

In [ ]:
# Сначала посмотрим на значения признака
data['decline_app_cnt'].value_counts().head(10)

Как видим, подавляющее количество клиентов не получало отказов по предыдущим заявкам.
Поэтому на основе этого признака создадим бинарный, показывающий, получал ли человек отказ ранее.

In [ ]:
data['decline_app_bin'] = data['decline_app_cnt'].apply(lambda g: int(g != 0))

# Добавляем признак в список бинарных
bin_cols.append('decline_app_bin')

Теперь создадим ещё один признак, прологарифмировав значения признака.
Сам признак оставим без изменений, потом создадим на его основе ещё пару признаков.

In [ ]:
data['decline_app_log'] = data['decline_app_cnt'].apply(lambda x: np.log(x + 1))

data['decline_app_log'].hist(bins=50)
plt.show()

quantile_info(data['decline_app_log'])

# Добавляем новый признак в список числовых
num_cols.append('decline_app_log')

Выбросы оставим, потому что иначе в столбце останутся одни нулевые значения

**Признак bki_request_cnt**

In [ ]:
# Сначала посмотрим на значения признака
data['bki_request_cnt'].value_counts().head(10)

Разница в количестве ненулевых и нулевых значений не такая большая, как в признаке decline_app_cnt.
Но, тем не менее, на основе этого признака создадим бинарный, показывающий, обращался ли человек в БКИ ранее. Может, это будет полезно для нашей модели.

In [ ]:
data['bki_request_bin'] = data['bki_request_cnt'].apply(lambda h: int(h != 0))

# Добавляем признак в список бинарных
bin_cols.append('bki_request_bin')

data['bki_request_bin'].value_counts()

Теперь создадим ещё один признак, прологарифмировав значения признака.
Сам признак оставим без изменений, потом создадим на его основе ещё пару признаков.

In [ ]:
data['bki_request_log'] = data['bki_request_cnt'].apply(lambda x: np.log(x + 1))

data['bki_request_log'].hist(bins=50)
plt.show()

quantile_info(data['bki_request_log'])

In [ ]:
# В столбце содержится 15 выбросов. Заменим их на меданные значения
data['bki_request_log'] = change_on_median(data['bki_request_log'])

# Добавляем новый признак в список числовых
num_cols.append('bki_request_log')

**Признак score_bki**

In [ ]:
# Посмотрим на характеристики распределения
data['score_bki'].hist(bins=50)
plt.show()

quantile_info(data['score_bki'])

In [ ]:
# Создадим признак, заменив выбросы на медианное значение
data['score_bki_clear'] = change_on_median(data['score_bki'])

# Добавляем новый признак в список числовых
num_cols.append('score_bki_clear')

**Признак income**

In [ ]:
# Прологарифмируем и посмотрим наличие выбросов
data['income'] = data['income'].apply(lambda x: np.log(x + 1))
data['income'].hist(bins=50)
plt.show()

quantile_info(data['income'])

In [ ]:
# Заменим все выбросы на медианное значение
data['income'] = change_on_median(data['income'])

In [ ]:
# создание признаков из информации в БКИ
data['uno'] = (data['bki_request_cnt'] - data['decline_app_cnt']) * data['score_bki']
data['uno_log'] = (data['bki_request_log'] - data['decline_app_log']) * data['score_bki_clear']

data['dos'] = data['bki_request_cnt'] * data['decline_app_cnt'] * data['score_bki']
data['dos_log'] = data['bki_request_log'] * data['decline_app_log'] * data['score_bki_clear']

# Добавляем новые признаки в список числовых
num_cols.append('uno')
num_cols.append('uno_log')
num_cols.append('dos')
num_cols.append('dos_log')

### Теперь обработаем категориальные признаки
Изначально к категориальным мы отнесли следующие признаки:

cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']

In [ ]:
# Посмотрим на них поближе:
data[cat_cols].head(10)

In [ ]:
# Как мы помним, в столбце education содержатся пропуски.
# Заполним их тем же способом, что и для наивной модели.
data['education'] = data['education'].fillna('SCH')

In [ ]:
# Посмотрим распределения категориальных переменных в выборке:
for e in cat_cols:
    data[e].value_counts(dropna=False).plot.barh()
    plt.title(e)
    plt.show()

К сожалению, без дополнительной информации мы не можем сделать выводов о содержании данных столбцов, поэтому оставим эти данные без обработки. Позже закодируем их с помощью OneHotEncoder.

### Теперь обработаем бинарные признаки
К бинарным мы отнесли следующие признаки:

bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [ ]:
# Посмотрим на них поближе:
data[bin_cols].head()

In [ ]:
# Переведём бинарные признаки в числа с помощью LabelEncoder
label_encoder = LabelEncoder()

for f in bin_cols:
    data[f] = label_encoder.fit_transform(data[f])
    
# убедимся в преобразовании    
data[bin_cols].head()

Объединим признаки car и car_type в один, категориальный:

0 - нет машины

1 - есть отечественная машина

2 - есть иномарка

In [ ]:
data['car'] = data['car'] + data['car_type']
data.drop('car_type', axis=1, inplace=True)
# Удалим признак из бинарных и переведём его в категориальные
bin_cols.remove('car_type')
bin_cols.remove('car')
cat_cols.append('car')

# Теперь оценим важность признаков и посмотрим матрицу корреляций

Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn. mutual_info_classif сначала требует, чтобы категориальные значения были сопоставлены с целочисленными значениями, поэтому необходимо преобразовать столбец education к целочисленным значениям

In [ ]:
edu_dict = {'ACD':0, 'PGR':1, 'UGR':2, 'GRD':3, 'SCH':4}
data['education'] = data['education'].apply(lambda x: edu_dict[x])
data['education'].value_counts(dropna=False)

In [ ]:
imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features=True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
# Cильная корреляция между переменными вредна для
# линейных моделей из-за неустойчивости полученных оценок.
# Оценим корреляцию Пирсона для численных переменных:
plt.figure(figsize=(14, 8))
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1, annot=True)
plt.show()

# Критерием для удаления признаков будем считать корреляцию выше 0,7

Для оценки значимости числовых переменных будем использовать функцию f_classif из библиотеки sklearn. 

Возможности модуля sklearn.feature_selection могут быть использованы не только для выбора важных признаков, но и для уменьшения размерности, улучшения предсказательной силы моделей, либо для повышения их производительности на очень многомерных наборах данных.

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t). 

В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [ ]:
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

На основании матрицы корреляций и оценки важности признаков удалим из датасета следующие столбцы:

weekday, uno,dos_log, uno_log, day, month, bki_request_cnt, decline_app_cnt, score_bki


In [ ]:
data.drop(['weekday', 'uno', 'dos_log', 'uno_log', 'day', 'month', 'bki_request_cnt', 'decline_app_cnt', 'score_bki'], axis=1, inplace=True)
data.info()

In [ ]:
num_cols.remove('uno')
num_cols.remove('uno_log')
num_cols.remove('dos_log')
num_cols.remove('day')
num_cols.remove('month')
num_cols.remove('bki_request_cnt')
num_cols.remove('decline_app_cnt')
num_cols.remove('score_bki')
cat_cols.remove('weekday')

In [ ]:
# Закодируем категориальные и бинарные признаки с помощью OneHotEncoding
data =  pd.get_dummies(data, columns=cat_cols+bin_cols)
data.info()

In [ ]:
# Выделим в выборке обучающую часть и тестовую по признаку sample
train_data = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим обучающую выборку на обучающую и валидационную части, для проверки разных моделей. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submission на kaggle.
Заодно старндартизируем числовые признаки**

In [ ]:
X_train, X_valid, Y_train, Y_valid = form(train_data)

In [ ]:
X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

# Модели логистической регрессии

### Модель логистической регрессии с параметрами по умолчанию

In [ ]:
# Обучаем модель на стандартных настройках логистической регрессии
model_simple = LogisticRegression(max_iter=5000)
model_simple.fit(X_train, Y_train)

probs_simple = model_simple.predict_proba(X_valid)
Y_pred_simple = model_simple.predict(X_valid)
probs_simple = probs_simple[:,1]

# Выводим на экран график roc_auc
print_roc_auc(Y_valid, probs_simple)

In [ ]:
confusion_matrix(Y_valid, Y_pred_simple)

In [ ]:
# Как видим, новая модель уже пытается различать классы.
# Посмотрим остальные метрики качества модели
print_metrics(Y_valid, Y_pred_simple)

Метрики качества очень плохи. Вспомним, что у нас несбалансированные классы: значений default=1 гораздо меньше, чем default=0. Построим новую модель с балансировкой классов

### Модель логистической регрессии с балансировкой классов

In [ ]:
# Обучаем модель логистической регрессии 
model_balance = LogisticRegression(class_weight='balanced', solver='liblinear',max_iter=5000)
model_balance.fit(X_train, Y_train)
probs_balance = model_balance.predict_proba(X_valid)
Y_pred_balance = model_balance.predict(X_valid)

probs_balance = probs_balance[:,1]


fpr, tpr, threshold = roc_curve(Y_valid, probs_balance)
roc_auc = roc_auc_score(Y_valid, probs_balance)

# Выводим на экран график roc_auc
print_roc_auc(Y_valid, probs_balance)

In [ ]:
confusion_matrix(Y_valid, Y_pred_balance)

In [ ]:
# Судя по матрице ошибок, новая модель работает лучше.
# Посмотрим другие метрики качества нашей модели
print_metrics(Y_valid, Y_pred_balance)

Действительно, метрики качества показали существенный рост.

Однако попробуем другие способы балансировки классов

# Модель с undersampling

Сравняем количество классов в обучающей выборке, удалив большое количество примеров с default=0

In [ ]:
data_under_1 = train_data[train_data['default'] == 1].copy()
count_1 = len(data_under_1)
data_under_0 = train_data[train_data['default'] == 0].iloc[:count_1,:].copy()
data_under = pd.concat([data_under_1, data_under_0])
# Проверим количество классов
data_under['default'].value_counts()

In [ ]:
# Разделим датасет на обучающий и валидационный
X_train_under, X_valid_under, Y_train_under, Y_valid_under = form(data_under)

In [ ]:
# Обучаем модель логистической регрессии
model_under = LogisticRegression(solver='liblinear', max_iter=5000)
model_under.fit(X_train_under, Y_train_under)

probs_under = model_under.predict_proba(X_valid_under)
Y_pred_under = model_under.predict(X_valid_under)
probs_under = probs_under[:,1]


fpr, tpr, threshold = roc_curve(Y_valid_under, probs_under)
roc_auc = roc_auc_score(Y_valid_under, probs_under)

# Выводим на экран график roc_auc
print_roc_auc(Y_valid_under, probs_under)

In [ ]:
confusion_matrix(Y_valid_under, Y_pred_under)

In [ ]:
# Метрики качества нашей модели
print_metrics(Y_valid_under, Y_pred_under)

У новой модели гораздо лучше определяются результаты TN.

Проверим ещё один способ работы с несбалансированными классами - метод oversampling.

Для этого увеличим количество примеров меньшего класса примерно до количества примеров большего класса, просто скопровав их несколько раз.

# Модель с oversampling

In [ ]:
# Сравняем количество классов в обучающей выборке, скопировав несколько раз примеры с default=1
data_over_1 = train_data[train_data['default'] == 1].copy()
count_1 = len(data_over_1)
data_over_0 = train_data[train_data['default'] == 0].copy()
count_0 = len(data_over_0)

M = round(count_0 / count_1)

df_list = [data_over_1]
df_list = df_list*M
df_list.append(data_over_0)

data_over = pd.concat(df_list).reset_index()
# Проверим количество классов

data_over['default'].value_counts()


In [ ]:
# Разделим датасет на обучающий и валидационный
X_train_over, X_valid_over, Y_train_over, Y_valid_over = form(data_over)

In [ ]:
# Обучаем модель логистической регрессии
model_over = LogisticRegression(solver='liblinear', max_iter=5000)
model_over.fit(X_train_over, Y_train_over)

probs_over = model_over.predict_proba(X_valid_over)
Y_pred_over = model_over.predict(X_valid_over)
probs_over = probs_over[:,1]

# Выводим на экран график roc_auc
print_roc_auc(Y_valid_over, probs_over)

In [ ]:
confusion_matrix(Y_valid_over, Y_pred_over)

In [ ]:
# Метрики качества нашей модели
print_metrics(Y_valid_over, Y_pred_over)

Показатели этой модели совсем чуть-чуть отличаются от предыдущей модели.

Несмотря на метрику ROC_AUC, другие метрики показали, что лучше определяют классы модели с undersampling и oversampling. Показатели у них примерно одинаковые, но oversampling требует значительно больше вычислительных ресурсов. Поэтому выбираем модель с undersampling как наилучшую.

### Работа с выбранной моделью

In [ ]:
# Проверим, не переобучилась ли наша модель
# Для этого сначала посчитаем метрики модели для тренировочной выборки
Y_pred_train_under = model_under.predict(X_train_under)
print('Метрики модели на тренировочных данных')
print_metrics(Y_train_under, Y_pred_train_under)

# А затем для для валидационной выборки
print('\nМетрики модели на валидационных данных')
Y_pred_valid_under = model_under.predict(X_valid_under)
print_metrics(Y_valid_under, Y_pred_valid_under)

Метрики почти не отличаются, значит, модель не переобучилась 

Теперь с помощью функции GridSearchCV найдём оптимальные гиперпараметры для нашей модели и посмотрим, улучшатся ли метрики

In [ ]:
# Выключим предупреждения
import warnings
warnings.filterwarnings("ignore")

model = LogisticRegression()

iter_ = 50
epsilon_stop = 1e-3

# Создадим список гиперпараметров
param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'saga'], 
     'class_weight': [None, 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter': [iter_],
     'tol': [epsilon_stop]},
    
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'saga'], 
     'class_weight': [None, 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter': [iter_],
     'tol': [epsilon_stop]},
]

gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=12, cv=5)
gridsearch.fit(X_train_under, Y_train_under)
model = gridsearch.best_estimator_

# Печатаем получившиеся наилучшие гиперпараметры
print('Наилучшие гиперпараметры:')
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))
        
# Печатаем метрики с этими наилучшими гиперпараметрами
preds = model.predict(X_valid_under)
y_pred_prob = model.predict_proba(X_valid_under)[:,1]
y_pred = model.predict(X_valid_under)

print('\nПолучившиеся метрики:')
print_metrics(Y_valid_under, y_pred)

Как видим, метрики практически не изменились. Тогда оставим гиперпараметры по умолчанию.

In [ ]:
# Подготовим тестовые данные для работы с моделью
# Удалим столбец default с фиктивными значениями
X_test_pd = test_data.drop(['default'], axis=1)

# Стандартизуем числовые данные по всем данным в тренировочном датасете
scaler_best = StandardScaler()
scaler_best.fit(train_data[num_cols].values)
X_test_num = scaler_best.transform(X_test_pd[num_cols].values)

# И соберём все признаки в numpy array
X_test = np.hstack([X_test_num, X_test_pd.drop(columns=num_cols).values])

# Создание файла submission

In [ ]:
test.head()

In [ ]:
Y_sub = model_under.predict_proba(X_test)
test['default'] = Y_sub[:,1]

In [ ]:
submission = test[['client_id','default']]
display(submission.head(10))
display(submission.shape)

In [ ]:
submission.to_csv('submission.csv', index=False)